<div style="width: 100%; clear: both;">
    <div style="float: left; width: 50%;">
       <img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
    </div>
</div>

<div style="float: right; width: 50%;">
    <p style="margin: 0; padding-top: 22px; text-align:right;">22.403 · Programació per a la ciència de dades</p>
    <p style="margin: 0; text-align:right;">Grau en Ciència de Dades Aplicada</p>
    <p style="margin: 0; text-align:right; padding-button: 100px;">Estudis d'Informàtica, Multimèdia i Telecomunicació</p>
</div>

</div>
<div style="width: 100%; clear: both;">
<div style="width:100%;">&nbsp;</div>

Programación para la ciencia de datos - PEC4
============================

En este Notebook encontraréis el ejercicio que supone la cuarta y última actividad de evaluación continuada (PEC) de la asignatura. Esta PEC intenta presetaros un pequeño proyecto en el cual debéis resolver diferentes ejercicios, que  engloba muchos de los conceptos cubiertos durante la asignatura. 

El objetivo de este ejercicio será desarrollar un **paquete de Python** fuera del entorno de Notebooks, que nos permita resolver el problema dado. 
Trabajaréis en archivos Python planos `.py`. Este tendrá que incluir el correspondiente código organizado lógicamente (separado por módulos, organizados por funcionalidad,...), la documentación del código (*docstrings*) y tests. Además, tendréis que incluir los correspondientes archivos de documentación de alto nivel (`README`) así como los archivos de licencia y dependencias (`requirements.txt`) comentados en la teoría.

Hacer un `setup.py` es opcional, pero si se hace se valorará positivamente de cara a la nota de la práctica y del curso.


# Enunciado:

La compañía de medios de comunicación Open Broadcast Corporation se está planteando adquirir licencias de emisión de programas de televisión populares con las que espera aumentar significativamente el número de suscriptores. Para poder tomar las decisiones de la mejor manera posible, nos han encargado analizar el contenido de la base de datos *The Movie Database (TMDB)*, que contiene información de más de 159.000 programas de televisión.
La información de esta base de datos está distribuida en tres ficheros que contienen las siguientes variables:

**TMDB_info.csv**
- **id**: identificador único de la serie
- **name**: nombre de la serie
- **number_of_seasons**: número de temporadas de la serie
- **number_of_episodes**: número de episodios de la serie
- **original_language**: idioma original
- **languages**: idiomas en que está disponible la serie
- **spoken_languages**: idiomas hablados en la serie
- **episode_run_time**: tiempo de duración de cada episodio en minutos
- **vote_count**: número de votos
- **vote_average**: valor promedio de los votos
- **popularity**: indicador de popularidad
- **first_air_date**: fecha de la primera emisión
- **last_air_date**: fecha de la última emisión
- **adult**: si el contenido de la serie es para adultos
- **in_production**: si la serie está en producción
- **type**: tipo de programa
- **status**: estado de la serie

**TMDB_overview.csv**
- **id**: identificador único de la serie
- **original_name**: nombre de la serie en su idioma original
- **tagline**: eslogan publicitario
- **overview**: resumen de la trama
- **backdrop_path**: ruta del backdrop de la serie en su página web
- **homepage**: página web de la serie
- **poster_path**: ruta del poster de la serie en su página web

**TMDB_distribution.csv**
- **id**: identificador único de la serie
- **genres**: géneros de la serie
- **created_by**: autores de la serie
- **networks**: plataformas de emisión
- **production_companies**: empresas productoras
- **origin_country**: países de origen
- **production_countries**: países de producción

Estos tres ficheros se encuentran en la carpeta comprimida **TMDB.zip**.


# Presentación de los resultados: 

Para hacer la entrega más fácil y homogénea os pedimos que organicéis el código de tal manera que **desde el fichero principal retorne todas las respuestas que se os pida en la PEC** haciendo uso de funciones que tendréis que definir en módulos.  Para eso, en cada ejercicio, os indicaremos el formato que tiene que tener cada respuesta, de tal manera que ejecutando `main.py` se vaya respondiendo a toda la PEC. Por defecto, `main.py` debe ejecutar todas las funciones de la PEC mostrando cómo funcionan pero también debe permitir ejecutarlas una a una si se desea. Debéis documentarlo todo muy bien en el README para que se pueda ejecutar sin problema. Os recordamos que en el README también tenéis que indicar como ejecutar los test y comprobar la cobertura de éstos.

### Ejercicio 1: Descompresión y lectura de ficheros.


#### Ejercicio 1.1.

Implementad una función que descomprima ficheros en formato zip y tar.gz. La función recibirá como inputs la ruta con el nombre del fichero que se quiere descomprimir. La función detectará automáticamente si el fichero está comprimido en zip o en tar.gz y mostrará un mensaje de error cuando el fichero sea de otro tipo. Utilizad esta función para descomprimir el fichero TMDB.zip.


#### Ejercicio 1.2.

Implementad una función que lea los csv y los integre en un único dataframe utilizando como clave la columna "id" utilizando la librería **pandas**. Obtened el tiempo de procesamiento.


#### Ejercicio 1.3.

Implementad una función que lea los csv y los integre en un único dataframe utilizando como clave la columna "id" utilizando la librería **csv**. Obtened el tiempo de procesamiento.

#### Ejercicio 1.4.

¿Qué diferencias se observan en la lectura de los ficheros siguiendo ambos métodos? ¿Si los ficheros tuvieran un tamaño de 10GB qué método sería más rápido? Justificad la respuesta.


### Ejercicio 2: Procesamiento de datos.


#### Ejercicio 2.1.

Añadid una variable air_days al dataframe que consista en el número de días que una serie ha estado en emisión. Mostrad por pantalla los 10 registros del dataset que más días han estado en emisión.


#### Ejercicio 2.2.

Cread un diccionario ordenado cuya clave será el nombre de la serie (name) y cuyo valor será la dirección web completa de su poster (homepage y poster_path). En caso de que homepage o poster_path tengan el valor NaN o "", el valor será el string “NOT AVAILABLE”. Mostrad por pantalla los primeros 5 registros del diccionario.


### Ejercicio 3: Filtrado de datos.


#### Ejercicio 3.1.

Obtened y mostrad por pantalla los nombres de las series cuyo idioma original (original_language) sea el inglés y en cuyo resumen (overview) aparezcan las palabras “mystery” o “crime”, sin tener en cuenta mayúsculas ni minúsculas.


#### Ejercicio 3.2.

Obtened una lista de las series que han empezado en 2023 y han sido canceladas. Mostrad por pantalla los primeros 20 elementos de esta lista.


#### Ejercicio 3.3.

Obtened un dataframe con los nombres, los nombres originales, las plataformas de emisión y las empresas productoras de todas las series cuyo idioma (variable languages) sea el japonés y mostrar los primeros 20 registros por pantalla. Nota: tened en cuenta que consideramos series en japonés también aquellas que tengan idiomas adicionales, por ejemplo, un registro con idioma “en, ja, ko” se incluiría.


### Ejercicio 4: Análisis gráfico.


#### Ejercicio 4.1.

Mostrad en un gráfico de barras el número de series por año de inicio.


#### Ejercicio 4.2.

Construid un gráfico de líneas que muestre el número de series de cada categoría de la variable “type” producidas en cada década desde 1940. ¿Qué cambios de tendencia se observan?


#### Ejercicio 4.3.

Obtened el número de series por género y mostrad el porcentaje respecto al total en un gráfico circular. Los géneros que representen menos del 1% del total se incluirán en la categoría "Other". Tened en cuenta que una serie que tenga más de un género deberá incluirse en todas las categorías en que esté clasificada y que las series con el campo "genres" vacío no se incluyen.


### Ejercicio 5: Conclusiones.


Redactad un breve informe que recopile las conclusiones obtenidas en el análisis realizado.

## Criterios de corrección

Esta PEC se valorará siguiendo los siguientes criterios:

* **Funcionalidad** (6 puntos): Se valorará que el código implemente todo lo que se pide.
    * Ejercicio 1 (1.5 puntos)
    * Ejercicio 2 (1 punto)
    * Ejercicio 3 (1.5 puntos)
    * Ejercicio 4 (1.5 puntos)
    * Ejercicio 5 (0.5 puntos)

* **Documentación** (0.5 puntos):  Todas las funciones de los ejercicios de esta PEC tendrán que estar debidamente documentadas utilizando docstrings (en el formato que prefiráis). 
* **Modularidad** (0.5 puntos): Se valorará la modularidad del código (tanto la organización del código en módulos como la creación de funciones). 
* **Estilo** (0.5 puntos): El código tiene que seguir la guía de estilo de Python (PEP8), exceptuando los casos donde hacerlo complique la legibilidad del código.
* **Tests** (1.5 puntos): El código tiene que contener una o diversas *suites* de tests que permitan comprobar que el código funciona correctamente, con un mínimo del 50% de cobertura.
* **Requeriments** (0.5 puntos): Tenéis que incluir un fichero de *requirements* que contenga la lista de librerías necesarias para ejecutar el código.
* **README** y **licencia** (0.5 puntos): Tenéis que añadir también un fichero README, que presente el proyecto y explique cómo ejecutarlo, así como la inclusión de la licencia bajo la que se distribuye el código (podéis escoger la que queráis).

### Importante

**Nota 1**: De la misma manera que en las PECs anteriores, los criterios transversales se valorarán de manera proporcional a la parte de funcionalidad implementada.

Por ejemplo, si el código sólo implementa la mitad de la PEC y la documentación está perfecta la puntuación correspondiente a documentación será de 0.25.

**Nota 2**: Es imprescindible que el paquete que libréis se ejecute correctamente en la máquina virtual y que el fichero de REAMDE explique claramente cómo ejecutar el código para generar los resultados pedidos. Además en el README tiene que explicarse también cómo se ejecutarán los test y cómo se comprueba su cobertura.

**Nota 3**: Entregad el paquete como un único archivo .zip que contenga sólo el código en el Registro de Evaluación Continua. **El código de Python tendrá que estar escrito en ficheros planos de Python.**